In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Dense,Input,Lambda
from tensorflow.keras.applications import MobileNetV2

In [14]:
batch_size=32
img_size=(128,128)
attack_dir = r"..\data\Detectedface\ImposterFace"
real_dir = r"..\data\Detectedface\ClientFace"
data_dir = r"..\data\Detectedface"

In [23]:
datagen=ImageDataGenerator(validation_split=0.2)

In [24]:
train_generator=datagen.flow_from_directory(data_dir,target_size=(128,128),batch_size=32,class_mode='binary',subset='training')
validation_generator=datagen.flow_from_directory(data_dir,target_size=(128,128),batch_size=32,class_mode="binary",subset="validation")

Found 10092 images belonging to 2 classes.
Found 2522 images belonging to 2 classes.


In [25]:
model=Sequential()
model.add(Input(shape=(128,128,3)))
model.add(Lambda(preprocess_input))
base=MobileNetV2(include_top=False,weights='imagenet',input_shape=(128,128,3))
base.trainable=False
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))



In [26]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
hist=model.fit(train_generator,validation_data=validation_generator,epochs=15,verbose=2)

Epoch 1/15


KeyboardInterrupt: 